# Position-Distance-Based Interaction Objective

In [2]:
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
plt.ioff()
from IPython.display import HTML

In [8]:
import mantrap
import numpy as np
import torch

env = mantrap.environment.PotentialFieldEnvironment(
    ego_type=mantrap.agents.IntegratorDTAgent, 
    ego_position=torch.zeros(2), 
    ego_velocity=torch.zeros(2),
    dt=0.4
)
env.add_ado(position=torch.tensor([-5, 2.0]), velocity=torch.tensor([1, 0]))

# Compute ego trajectories to base comparison on.
t_horizon = 9
ego_controls = [
    torch.zeros((t_horizon, 2), requires_grad=True),
    torch.tensor([[0.0, -1.0] * t_horizon], requires_grad=True).view(t_horizon, 2),
    torch.tensor([[1.0, -1.0] * t_horizon], requires_grad=True).view(t_horizon, 2),
    torch.tensor([[-1.0, 0.0] * t_horizon], requires_grad=True).view(t_horizon, 2),
    torch.tensor([[0.0, 1.0] * t_horizon], requires_grad=True).view(t_horizon, 2),
]
colors = ["red", "blue", "green", "yellow", "orange"]
assert len(ego_controls) == len(colors)

n = len(ego_controls)
ego_trajectories = [env.ego.unroll_trajectory(controls, dt=env.dt) for controls in ego_controls]

# Compute trajectory predictions in response to different ego actions.
ado_trajectory_wo = env.predict_wo_ego(t_horizon=t_horizon + 1).squeeze()
ado_trajectories_w = [env.predict_w_trajectory(trajectory).squeeze() for trajectory in ego_trajectories]

In [9]:
from mantrap.visualization import visualize_trajectories

labels_all = ["ado_wo", *[f"ado_{i + 1}" for i in range(n)], *[f"ego_{i + 1}" for i in range(n)]]
colors_all = ["black", *colors, *colors]

HTML(visualize_trajectories([ado_trajectory_wo, *ado_trajectories_w, *ego_trajectories],
                             labels=labels_all, colors=colors_all, env_axes=env.axes))

In [12]:
cost = mantrap.modules.baselines.InteractionPositionModule(env=env, t_horizon=t_horizon)

print("Objectives")
for trajectory, color in zip(ego_trajectories, colors):
    objective = cost.objective(trajectory, ado_ids=env.ado_ids, tag="test")
    print(f"{color} ==> {objective}")
    
print("\nGradients (L2-norm)")
for controls, trajectory, color in zip(ego_controls, ego_trajectories, colors):
    gradient = np.linalg.norm(cost.gradient(trajectory, grad_wrt=controls, ado_ids=env.ado_ids, tag="test"))
    print(f"{color} ==> {gradient}")

Objectives
red ==> 0.08563081175088882
blue ==> 0.20239576697349548
green ==> 0.04896272346377373
yellow ==> 0.17392438650131226
orange ==> 0.2659914493560791

Gradients (L2-norm)
red ==> 0.09069831669330597
blue ==> 0.23021291196346283
green ==> 0.09155204892158508
yellow ==> 0.23476864397525787
orange ==> 0.2039763182401657


Comment: A planning horizon `t_planning <= 2` will not affect the interaction objective, given some environment which models the ados as double integrators, such as the `PotentialFieldEnvironment` used in this example. This is due to the fact, that the position of a double integrator merely is affected by some force acting on it one  time-step later (see double integrator dynamics).

### Probability-Based Interaction Objective

In [14]:
cost = mantrap.modules.InteractionProbabilityModule(env=env, t_horizon=t_horizon, weight=1.0)

print("Objectives")
for trajectory, color in zip(ego_trajectories, colors):
    objective = cost.objective(trajectory, ado_ids=env.ado_ids, tag="test")
    print(f"{color} ==> {objective}")
    
print("\nGradients (L2-norm)")
for controls, trajectory, color in zip(ego_controls, ego_trajectories, colors):
    gradient = np.linalg.norm(cost.gradient(trajectory, grad_wrt=controls, ado_ids=env.ado_ids, tag="test"))
    print(f"{color} ==> {gradient}")

Objectives
red ==> 36.68547058105469
blue ==> 31.778337478637695
green ==> 32.238285064697266
yellow ==> 35.06740951538086
orange ==> 36.64483642578125

Gradients (L2-norm)
red ==> 4.339778900146484
blue ==> 11.726958274841309
green ==> 6.827750205993652
yellow ==> 11.09294319152832
orange ==> 9.528249740600586
